# Events of interest from raw instrumentation logs

In [1]:
import pandas as pd

logs_df = pd.read_csv('loggedfs_events_pagerank.log')
cols = [col for col in logs_df.columns if col in ['Action','Time','Path','PID','PPID','UID','Command Line']]
df = logs_df[cols]
df[(df['Action'] == 'read') | (df['Action'] == 'write')]

,Time,Action,Path,UID,PID,PPID,Command Line
1,1490369977685,read,/home/lubuntu/src/file_access_monitor/test_wor...,1000,3013,3012,python concat_csvs.py in1.csv in2.csv out2.csv...
3,1490369977690,read,/home/lubuntu/src/file_access_monitor/test_wor...,1000,3013,3012,python concat_csvs.py in1.csv in2.csv out2.csv...
5,1490369977692,read,/home/lubuntu/src/file_access_monitor/test_wor...,1000,3013,3012,python concat_csvs.py in1.csv in2.csv out2.csv...
8,1490369977696,write,/home/lubuntu/src/file_access_monitor/test_wor...,1000,3013,3012,python concat_csvs.py in1.csv in2.csv out2.csv...
10,1490369979040,read,/home/lubuntu/src/file_access_monitor/test_wor...,1000,3020,3012,python concat_csvs.py in1.csv in2.csv out2.csv...
12,1490369979042,read,/home/lubuntu/src/file_access_monitor/test_wor...,1000,3020,3012,python concat_csvs.py in1.csv in2.csv out2.csv...
14,1490369979044,read,/home/lubuntu/src/file_access_monitor/test_wor...,1000,3020,3012,python concat_csvs.py in1.csv in2.csv out2.csv...
17,1490369979049,write,/home/lubuntu/src/file_access_monitor/test_wor...,1000,3020,3012,python concat_csvs.py in1.csv in2.csv out2.csv...


# Build a Neo4j lineage graph from instrumentation logs

In [4]:
import lineage_graph as lg

graph = lg.build_lineage_graph('loggedfs_events_pagerank.log')

# Custom visualization of lineage graph using Vis.js

In [5]:
from scripts.vis import draw

options = {"Dataset": "name", "Job": "uid"}
draw(graph, options)

# Query Neo4j temporal lineage graph using Cypher

## Query results as a Pandas DataFrame

In [ ]:
# Sanity check query.
query = """
    MATCH (n)-[r]->(m)
    RETURN id(n) AS source_id,
           id(r) AS rel_id,
           id(m) AS target_id
"""

results_df = pd.DataFrame(graph.data(query))
print(results_df)

## Copy detection as a reachability query between 2 content similar datasets

In [ ]:
query = """
    MATCH (n:Dataset)
    WHERE (n.name = '/home/lubuntu/src/file_access_monitor/test_workflows/in1.csv'
    OR n.name = '/home/lubuntu/src/file_access_monitor/test_workflows/out4.csv')
    RETURN id(n) as node_id
"""
node_ids = graph.data(query)

# Path query with monotonically increasing timestamps.
query = """
    START a=node(%d), b=node(%d)
    MATCH p=(a)-[r*]->(b)
    WITH head(relationships(p)) as r1, p, b
    RETURN p, b.name
""" % (node_ids[0]['node_id'], node_ids[1]['node_id'])


paths = graph.data(query)
print('\nFound %d path(s) from a to b:\n' % len(paths))
print(paths)

# Path query with monotonically increasing timestamps.
query = """
    START a=node(%d), b=node(%d)
    MATCH p=(b)-[r*]->(a)
    WITH head(relationships(p)) as r1, p, b
    RETURN p, b.name
""" % (node_ids[0]['node_id'], node_ids[1]['node_id'])


paths = graph.data(query)
print('\nFound %d path(s) from b to a:\n' % len(paths))
print(paths)

## Copy detection as a reachability query between 2 content similar datasets (monotonically increasing edge timestamps)

In [ ]:
query = """
    MATCH (n:Dataset)
    WHERE (n.name = '/home/lubuntu/src/file_access_monitor/test_workflows/in1.csv'
    OR n.name = '/home/lubuntu/src/file_access_monitor/test_workflows/out4.csv')
    RETURN id(n) as node_id
"""
node_ids = graph.data(query)

# Path query with monotonically increasing timestamps.
query = """
    START a=node(%d), b=node(%d)
    MATCH p=(a)-[r*]->(b)
    WITH head(relationships(p)) as r1, p, b
    WHERE all(r2 in relationships(p)
              where r2.timestamp>=r1.timestamp)    
    RETURN p, b.name
""" % (node_ids[0]['node_id'], node_ids[1]['node_id'])


paths = graph.data(query)
print('\nFound %d path(s) from a to b with monotonically increasing timestamps:\n' % len(paths))
print(paths)

# Path query with monotonically increasing timestamps.
query = """
    START a=node(%d), b=node(%d)
    MATCH p=(b)-[r*]->(a)
    WITH head(relationships(p)) as r1, p, b
    WHERE all(r2 in relationships(p)
              where r2.timestamp>=r1.timestamp)    
    RETURN p, b.name
""" % (node_ids[0]['node_id'], node_ids[1]['node_id'])


paths = graph.data(query)
print('\nFound %d path(s) from b to a with monotonically increasing timestamps:\n' % len(paths))
print(paths)

# Aggregate Queries


In [ ]:
query = """
    START from=node(*)
    MATCH p=(from)-->(to)
    WITH from as from, to as to, count(p) as paths
    WHERE paths >= 1
    RETURN to,paths
"""

paths = graph.data(query)
print(paths)

# PageRank

In [7]:
%load_ext cypher
import networkx as nx
%matplotlib inline

results = %cypher MATCH p = (a)-[r*]->(b) RETURN p

# Networkx graph.
g = results.get_graph()

#nx.draw(g)
#g.nodes(data=True)

# Print nodes so that we can see their original ids and properties.
print("\nOriginal Nodes:")
print(g.nodes(data=True))

# Node weights for personalized pagerank.
personalize = {}
for node, data in g.nodes(data=True):
    if 'Dataset' in data['labels']:
        personalize[node] = 0
    elif 'Job' in data['labels']:
        # Let's weight each edge by the amount of CPU consumed.
        personalize[node] = data['cpu']
    print("node=%s, data=%s" % (node, data))

print("\nPersonalize Vector:")
print(personalize)

# Transformation from MultiDigraph to Graph for Pagerank calculation.
H = nx.Graph()
for src, dst, edge in g.edges(data=True):
    # Let's weight each edge by the amount of bytes read / written.
    w = edge['data_bytes']
    if H.has_edge(src, dst):
        H[src][dst]['weight'] += w
    else:
        H.add_edge(src, dst, weight=w)

print("\nPageRank:")
print(nx.pagerank(H))
print("\nPersonalized PageRank:")
print(nx.pagerank(H, personalization=personalize))

The cypher extension is already loaded. To reload it, use:
  %reload_ext cypher
14 rows affected.

Original Nodes:
[('109', {'uid': 'PID=3013,CLI=python concat_csvs.py in1.csv in2.csv out2.csv out3.csv ', 'labels': ['Job'], 'name': 'python concat_csvs.py in1.csv in2.csv out2.csv out3.csv ', 'cpu': 85.04303955809714, 'pid': 3013}), ('108', {'labels': ['Dataset'], 'name': '/home/lubuntu/src/file_access_monitor/test_workflows/in1.csv'}), ('113', {'uid': 'PID=3020,CLI=python concat_csvs.py in1.csv in2.csv out2.csv out4.csv ', 'labels': ['Job'], 'name': 'python concat_csvs.py in1.csv in2.csv out2.csv out4.csv ', 'cpu': 54.478126156204524, 'pid': 3020}), ('110', {'labels': ['Dataset'], 'name': '/home/lubuntu/src/file_access_monitor/test_workflows/in2.csv'}), ('112', {'labels': ['Dataset'], 'name': '/home/lubuntu/src/file_access_monitor/test_workflows/out3.csv'}), ('114', {'labels': ['Dataset'], 'name': '/home/lubuntu/src/file_access_monitor/test_workflows/out4.csv'}), ('111', {'labels': ['Da